## Evaluating a Regression Model

In this exercise, you will create a pipeline for a linear regression model, and then test and evaluate the model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [2]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", StringType(), False),
  StructField("DestAirportID", StringType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
  StructField("Late", IntegerType(), False),
])

data = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/flights.csv', schema=flightSchema, header=True)
data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a regression model

In [4]:
# Define the pipeline
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")
lr = LinearRegression(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

# Train the model
piplineModel = pipeline.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [6]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "label")
predicted.show()

+--------------------+-------------------+-----+
 features| prediction|label|
+--------------------+-------------------+-----+
[25.0,2.0,10.0,1....| 91.52128720864461| 113|
[25.0,2.0,10.0,1....| 3.5846973888148455| 2|
[25.0,2.0,10.0,57...| 20.38819794586432| 41|
[25.0,2.0,10.0,11...| 38.14372429181161| 30|
[25.0,2.0,10.0,49...| 14.932194625977907| 7|
[25.0,2.0,10.0,49...| 93.14880725223644| 82|
[25.0,2.0,10.0,37...| 30.23377016918674| 27|
[25.0,2.0,10.0,37...|-0.4819033034982638| 62|
[25.0,2.0,10.0,37...|-1.3598588409029375| -10|
[25.0,2.0,10.0,37...| 53.35888422463853| 44|
[25.0,2.0,10.0,37...| -2.390329802958256| -19|
[25.0,2.0,10.0,23...| 57.74818192173646| 50|
[25.0,2.0,10.0,14...| -3.551366103062641| -15|
[25.0,2.0,10.0,48...| -3.195107989842697| 28|
[25.0,2.0,10.0,48...| -9.645827600976709| -14|
[25.0,2.0,10.0,12...|-1.4096893022548542| -20|
[25.0,2.0,10.0,12...| 60.70244878273547| 83|
[25.0,2.0,10.0,16...|-0.7549186041008795| -2|
[25.0,2.0,10.0,17...| 63.314576488268244| 29|
[25.0,2.0,2.0,34....| 16.69998763983852| 6|
+--------------------+-------------------+-----+
only showing top 20 rows

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot to see how well the predicted delay correlates to the actual delay.

In [8]:
predicted.createOrReplaceTempView("regressionPredictions")

In [9]:
%sql
SELECT label, prediction FROM regressionPredictions

label,prediction
113,91.52128720864461
2,3.5846973888148455
41,20.38819794586432
30,38.14372429181161
7,14.932194625977907
82,93.14880725223644
27,30.23377016918674
62,-0.4819033034982638
-10,-1.3598588409029375
44,53.35888422463853


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the predicted and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 17.30038849531777